#### Data location
#### https://valuation.property.nsw.gov.au/embed/propertySalesInformation
#### http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181001.zip 
#### http://www.valuergeneral.nsw.gov.au/__psi/yearly/2017.zip

### Metadata
http://www.valuergeneral.nsw.gov.au/__data/assets/pdf_file/0015/216402/Current_Property_Sales_Data_File_Format_2001_to_Current.pdf
http://www.valuergeneral.nsw.gov.au/__data/assets/pdf_file/0016/216403/Property_Sales_Data_File_-_Data_Elements_V3.pdf

In [1]:
import zipfile
import pandas as pd
#import matplotlib.pyplot as plt
from datetime import datetime
import collections
import requests
import os
import requests
import io
import zipfile
import datetime
import time 
import glob
import csv

#### Weekly load

In [3]:
def load_sales_data(file_path, folder_level=-2):
    dat_files = [f for f in glob.glob("{0}*.DAT".format(file_path))]
    foldername = (file_path.split("/")[folder_level])
    print("Loading sales data for the date - " + foldername)
    try:
        os.remove("../data/sales/A_str/{0}.csv".format(foldername))
        os.remove("../data/sales/B_str/{0}.csv".format(foldername))
        os.remove("../data/sales/C_str/{0}.csv".format(foldername))
        os.remove("../data/sales/D_str/{0}.csv".format(foldername))
        os.remove("../data/sales/Z_str/{0}.csv".format(foldername))
    except OSError:
        pass
    for i in dat_files:
        A_str = ""
        B_str = ""
        C_str = ""
        D_str = ""
        Z_str = ""
        with open(i) as f:
            lis=[line.split() for line in f]
            for i in lis:
                if ("".join(i)).split(";")[0] == 'A':
                    A_str =  A_str + ";".join(("".join(i)).split(";")) + "\n"
                if ("".join(i)).split(";")[0] == 'B':
                    B_str = B_str + ";".join(("".join(i)).split(";")) + "\n"
                if ("".join(i)).split(";")[0] == 'C':
                    C_str = C_str + ";".join(("".join(i)).split(";")) + "\n"
                if ("".join(i)).split(";")[0] == 'D':
                    D_str = D_str + ";".join(("".join(i)).split(";")) + "\n"
                if ("".join(i)).split(";")[0] == 'Z':
                    Z_str = Z_str + ";".join(("".join(i)).split(";")) + "\n"
        with open("../data/sales/A_str/{0}.csv".format(foldername), "a") as a_myfile:
            a_myfile.write(A_str)
        with open("../data/sales/B_str/{0}.csv".format(foldername), "a") as b_myfile:
            b_myfile.write(B_str)
        with open("../data/sales/C_str/{0}.csv".format(foldername), "a") as c_myfile:
            c_myfile.write(C_str)
        with open("../data/sales/D_str/{0}.csv".format(foldername), "a") as d_myfile:
            d_myfile.write(D_str)
        with open("../data/sales/Z_str/{0}.csv".format(foldername), "a") as z_myfile:
            z_myfile.write(Z_str)

def download_extract_zip(url):
    response = requests.get(url)
    if (response.status_code==200):
        with zipfile.ZipFile(io.BytesIO(response.content)) as thezip:
            path2download = "../data/temp/{0}/".format((url.split("/")[-1]).replace(".zip",""))
            thezip.extractall(path2download)
            print("Files are downloaded successfully in {0} !!!".format(path2download))
            load_sales_data(path2download)
            #return path2download
    else:
        print("No data found")
#download_extract_zip(url)                



def property_price_sq_meter(row):
    if row['Area_Type'] == 'M':
        return  row['Purchase_Price'] / row['Area']
    if row['Area_Type'] =='H':
        return  row['Purchase_Price'] / (row['Area'] * 10000)


    
def sale_datamart(year):    
    col_header =['Record_Type',
    'District_Code',
    'Property_Id',
    'Sale_Counter',
    'Download_Date_Time',
    'Property_Name',
    'Property_Unit_Number',
    'Property_House_Number',
    'Property_Street_Name',
    'Property_Locality',
    'Property_Post_Code',
    'Area',
    'Area_Type',
    'Contract_Date',
    'Settlement_Date',
    'Purchase_Price',
    'Zoning',
    'Nature_of_Property',
    'Primary_Purpose',
    'Strata_Lot_Number',
    'Component_code',
    'Sale_Code',
    'Percentage_Interest_of_Sale',
    'Dealing_Number',
    'Empty']
    
    print("Loading datamart .. "  + '../data/sales/B_str/{0}.csv'.format(year))
    lst_pds = [pd.read_csv(f, 
                           sep=';', 
                           error_bad_lines=False, 
                           quoting=csv.QUOTE_NONE, 
                           header=None ,  
                           encoding='utf-8',
                           index_col=None
                          ) for f in glob.glob('../data/sales/B_str/{0}.csv'.format(year))]
    
    df_sale = pd.concat(lst_pds)
    df_sale.columns = col_header
    # Add Downloaded_Date from 
    df_sale['Download_Date'] = pd.to_datetime(df_sale['Download_Date_Time'].str[:8]) 
    
    #Deduplicate 
    df_sale_dedup = df_sale.drop_duplicates(subset=[i for i in df_sale.columns if i != 'Download_Date'], keep=False)
    df_sale_dedup['Property_Price_Sq_Meter'] = df_sale_dedup.apply(property_price_sq_meter, axis=1)
    
    # Build Data mart
    cols_lst     = ['Download_Date','Property_Locality', "Property_Post_Code"]
    cols_lst_key = ["Property_Price_Sq_Meter", "Purchase_Price"]
    agg_lst      = ['mean','count', 'min', 'max', 'median','std']
    
    df_time_ser_price = df_sale_dedup[(df_sale.Nature_of_Property == 'R')]\
                                        [cols_lst + cols_lst_key ]\
                                            .groupby(cols_lst)\
                                            .agg(agg_lst)
    
    df_time_ser_price =  df_time_ser_price.reset_index()
    df_time_ser_price.columns = cols_lst + [grp + '_' + agg for grp in cols_lst_key for agg in agg_lst]
    print(len(df_time_ser_price))
    # Export to csv file 
    df_time_ser_price.to_csv('../data/sales/sales_{}_data_mart.csv'.format(year),index=False)

In [3]:
# Weekly load 
days_to_load = 10


ts = time.time()
for i in range(0, days_to_load):
    a = str(datetime.date.today() - datetime.timedelta(i)).replace("-","")
    url = 'http://www.valuergeneral.nsw.gov.au/__psi/weekly/{0}.zip'.format(a)
    print(url)
    download_extract_zip(url)
te = time.time()
print(te-ts)
#185.92472124099731

http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181010.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181009.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181008.zip
Files are downloaded successfully in ../data/temp/20181008/ !!!
Loading sales data for the date - 20181008
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181007.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181006.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181005.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181004.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181003.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181002.zip
No data found
http://www.valuergeneral.nsw.gov.au/__psi/weekly/20181001.zip
Files are downloaded successfully in ../data/temp/20181001/ !!!
Loading sales data for the date - 20181001
9.229669094085693


In [4]:
sale_datamart("2018*")

Loading datamart .. ../data/sales/B_str/2018*.csv
40166


### History Load

In [10]:
## Yearly load - time delta for [2017,2016,2015] = 491.8507146835327
#ts = time.time()
#
#for yr in range(2017,2018):
#    url = 'http://www.valuergeneral.nsw.gov.au/__psi/yearly/{0}.zip'.format(yr)
#    print(url)
#    download_extract_zip(url)
#    
#    ## Extract weekly zip files 
#    dat_files = [f for f in glob.glob("../data/temp/{0}/*.zip".format(yr))]
#    for file in dat_files:
#        with zipfile.ZipFile(file) as thezip:
#            thezip.extractall("../data/temp/{0}/".format(yr))
#    
#    # Load yearly data 
#    # For 2015 alone - since it follows different folder struture
#    if (yr in [2015]):
#        load_sales_data("../data/temp/{0}/*/".format(yr),-3)
#    if (yr in [2016,2017]):
#        load_sales_data("../data/temp/{0}/".format(yr),-2)
#    else:
#        load_sales_data("../data/temp/{0}/{0}/*/".format(yr),-3)
#te = time.time()
#print(te-ts)



### Data Mart Load

In [5]:
## History Load 
for yr in range(2013,2018):
    print(yr)
    sale_datamart(yr)

2013
Loading datamart .. ../data/sales/B_str/2013.csv
49836
2014
Loading datamart .. ../data/sales/B_str/2014.csv
52925
2015
Loading datamart .. ../data/sales/B_str/2015.csv
40883
2016
Loading datamart .. ../data/sales/B_str/2016.csv
53492
2017
Loading datamart .. ../data/sales/B_str/2017.csv
55046


In [18]:
# Clean up temp space
#! rm -r ../data/temp/*
#! rm -r ../data/sales/*/'*.csv'

rm: cannot remove '../data/temp/*': No such file or directory
rm: cannot remove '../data/sales/*/*.csv': No such file or directory
